# Optimize with Hyperopt
[Home](https://github.com/hyperopt/hyperopt) | 
[Document](https://hyperopt.github.io/hyperopt/)

## Installation


In [ ]:
!pip install hyperopt

## Sample Strategy

In [1]:
# import talib.abstract as ta

from lettrade import DataFeed, Strategy, crossover, crossunder
from lettrade.exchange.backtest import ForexBackTestAccount, let_backtest
from lettrade.indicator.vendor.qtpylib import inject_indicators

inject_indicators()


class SmaCross(Strategy):
    ema1_period = 9
    ema2_period = 21

    def indicators(self, df: DataFeed):
        # df["ema1"] = ta.EMA(df, timeperiod=self.ema1_period)
        # df["ema2"] = ta.EMA(df, timeperiod=self.ema2_period)
        df["ema1"] = df.close.ema(window=self.ema1_period)
        df["ema2"] = df.close.ema(window=self.ema2_period)

        df["signal_ema_crossover"] = crossover(df.ema1, df.ema2)
        df["signal_ema_crossunder"] = crossunder(df.ema1, df.ema2)

    def next(self, df: DataFeed):
        if len(self.orders) > 0 or len(self.trades) > 0:
            return

        if df.l.signal_ema_crossover[-1]:
            price = df.l.close[-1]
            self.buy(size=0.1, sl=price - 0.001, tp=price + 0.001)
        elif df.l.signal_ema_crossunder[-1]:
            price = df.l.close[-1]
            self.sell(size=0.1, sl=price + 0.001, tp=price - 0.001)


lt = let_backtest(
    strategy=SmaCross,
    datas="example/data/data/EURUSD_5m_0_10000.csv",
    account=ForexBackTestAccount,
    # plotter=None,
)

## Optimize

In [2]:
# define a search space
from hyperopt import fmin, tpe, space_eval, Trials
from hyperopt import hp


lettrade_model = lt.optimize_model(fork_data=True)


def train_model(params):
    # Params
    params = [
        ("ema1_period", params["ema1_period"]),
        ("ema2_period", params["ema2_period"]),
    ]

    # Model
    result = lettrade_model(params)

    # Score
    return -result["equity"]


# Hyperopt
search_space = {
    "ema1_period": hp.uniform("ema1_period", 5, 25),
    "ema2_period": hp.uniform("ema2_period", 10, 50),
}

trials = Trials()
best_params = fmin(
    train_model,
    search_space,
    algo=tpe.suggest,
    max_evals=100,
    trials=trials,
)

100%|██████████| 1000/1000 [02:53<00:00,  5.76trial/s, best loss: -1189.18]


In [3]:
hyperparams = space_eval(search_space, best_params)

print(best_params)
print(hyperparams)
trials

{'ema1_period': 24.972284713859786, 'ema2_period': 11.593172228900498}
{'ema1_period': 24.972284713859786, 'ema2_period': 11.593172228900498}


## Plot
[Tutorial](https://medium.com/doma/visualizing-hyperparameter-optimization-with-hyperopt-and-plotly-states-title-26369b020b5b)

## Init Plotly environment

In [21]:
import plotly.io as pio

pio.renderers.default = "notebook"
pio.templates.default = "plotly_dark"

In [25]:
import pandas as pd
import numpy as np


def unpack(x):
    if x:
        return x[0]
    return np.nan


# We'll first turn each trial into a series and then stack those series together as a dataframe.
df = pd.DataFrame([pd.Series(t["misc"]["vals"]).apply(unpack) for t in trials])
# Then we'll add other relevant bits of information to the correct rows and perform a couple of
# mappings for convenience
df["loss"] = [t["result"]["loss"] for t in trials]
df["trial_number"] = df.index


df["win"] = -df["loss"]
df

,ema1_period,ema2_period,loss,trial_number,win
0,9.061579,26.491877,-842.38,0,842.38
1,23.485567,36.428490,-947.78,1,947.78
2,17.920396,15.558605,-1164.78,2,1164.78
3,9.309459,17.427338,-909.48,3,909.48
4,10.244380,17.155087,-950.28,4,950.28
...,...,...,...,...,...
995,20.632502,10.728158,-976.08,995,976.08
996,23.387091,14.726145,-1059.68,996,1059.68
997,24.654661,19.729115,-1029.08,997,1029.08
998,22.128256,12.305837,-1158.38,998,1158.38


### Type 1

In [26]:
from plotly import express as px
from plotly import graph_objects as go
from plotly import offline as pyo

fig = px.scatter(df, x="trial_number", y="win")
fig.show()

### Type 2

In [27]:
import plotly.express as px


fig = px.density_contour(
    df,
    x="ema1_period",
    y="ema2_period",
    z="win",
    histfunc="max",
)
fig.update_traces(contours_coloring="fill", contours_showlabels=True)
fig.show()

### Type 3

In [28]:
import plotly.express as px

fig = px.density_heatmap(
    df,
    x="ema1_period",
    y="ema2_period",
    z="win",
    nbinsx=20,
    nbinsy=40,
    histfunc="max",
    color_continuous_scale="Viridis",
)
fig.show()